<p><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"></p>
<h1 id="unSkript-Runbooks-&para;">unSkript Runbooks <a class="jp-InternalAnchorLink" href="#-unSkript-Runbooks-" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#unSkript-Runbooks-&para;" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success"><strong>&nbsp;This runbook demonstrates How to Update Tags for AWS Resources using unSkript legos.</strong></div>
<p>&nbsp;</p>
<center>
<h2 id="Enforce-Mandatory-Tags-Across-All-AWS-Resources&para;">Update Tags for selected AWS Resources<a class="jp-InternalAnchorLink" href="#Enforce-Mandatory-Tags-Across-All-AWS-Resources" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Enforce-Mandatory-Tags-Across-All-AWS-Resources&para;" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Steps-Overview&para;" target="_self">&para;</a></h1>
<ol>
<li>List all the Resources ARNs in the given region with the selected tag.</li>
<li>WE'll print a list of tagged resources along with the current value of the tag. Select and change as desired.</li>
<li>Update the Selected tags at AWS.</li>
</ol>

In [29]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_resources_with_tag_printer(output):
    if output is None:
        return
    pprint.pprint(f"there are {len(output)} resources with tag {Tag_Key}. We can fix a max of 20." )


@beartype
def aws_get_resources_with_tag(handle, region: str, tag:str) -> List:
    """aws_get_resources_with_tag Returns an List of Untagged Resources.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: Region to filter resources.

        :rtype: List of untagged resources.
    """

    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = []


    try:
        response = aws_get_paginator(ec2Client, "get_resources", "ResourceTagMappingList")
        for resources in response:
            if  resources["Tags"]:
                #has tags
                #print(tagged_instance)
                #get all the keys for the instance
                for kv in resources['Tags']:
                    key = kv["Key"]
                    if tag == key:
                        temp = {"arn": resources["ResourceARN"], "value":kv["Value"]}
                        result.append(temp)

    except Exception as error:
        result.append({"error":error})

    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "tag": "Tag_Key"
    }''')
task.configure(outputName="taggedResources")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_resources_with_tag, lego_printer=aws_get_resources_with_tag_printer, hdl=hdl, args=args)

In [54]:
import ipywidgets as widgets
from IPython.display import display

# List of dictionaries with ARN and value
data = taggedResources

# Create an HBox for each dictionary entry with checkbox and text field
checkboxes = []
text_fields = []
for entry in data:
    checkbox = widgets.Checkbox(value=False, description=entry['arn'])
    text_field = widgets.Text(description='Value:', value=entry['value'])
    hbox = widgets.HBox([checkbox, text_field])
    checkboxes.append(checkbox)
    text_fields.append(text_field)

# Output list of dictionaries
updated_list = []

# Function to handle checkbox change
def checkbox_changed(change):
    checkbox = change.owner
    index = checkboxes.index(checkbox)
    entry = data[index]
    entry['checked'] = checkbox.value
    update_output()

# Function to handle text field change
def text_field_changed(change):
    text_field = change.owner
    index = text_fields.index(text_field)
    entry = data[index]
    entry['value'] = text_field.value
    update_output()

# Register the checkbox change handler for each checkbox
for checkbox in checkboxes:
    checkbox.observe(checkbox_changed, 'value')

# Register the text field change handler for each text field
for text_field in text_fields:
    text_field.observe(text_field_changed, 'value')

# Function to update the output list of dictionaries
def update_output():
    global updated_list
    updated_list = [{'arn': entry['arn'], 'value': entry['value']} for entry in data if 'checked' in entry and entry['checked']]

# Display the checkboxes and text fields
checkboxes_widgets = []
for hbox in zip(checkboxes, text_fields):
    checkboxes_widgets.append(widgets.HBox(hbox))
display(*checkboxes_widgets)




In [55]:
print(output)

Here we will use unSkript AWS Attach Tags to Resources Lego. This lego take handle, resource_arn: list, tag_key: str, tag_value: str, region: str as input. This input is used to attach mandatory tags to all untagged Resources.

In [149]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field

from beartype import beartype
@beartype
def aws_attach_tags_to_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_attach_tags_to_resources(
    handle,
    resource_arn: list,
    tag_key: str,
    tag_value: str,
    region: str
    ) -> Dict:
    """aws_attach_tags_to_resources Returns an Dict of resource info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type resource_arn: list
        :param resource_arn: Resource ARNs.

        :type tag_key: str
        :param tag_key: Resource Tag Key.

        :type tag_value: str
        :param tag_value: Resource Tag value.

        :type region: str
        :param region: Region to filter resources.

        :rtype: Dict of resource info.
    """
    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = {}
    try:
        response = ec2Client.tag_resources(
            ResourceARNList=resource_arn,
            Tags={tag_key: tag_value}
            )
        result = response

    except Exception as error:
        result["error"] = error

    return result



task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "region": "Region",
    "resource_arn": "checked_list",
    "tag_key": "Tag_Key",
    "tag_value": "\\"01/01/2025\\""
    }''')
task.configure(iterJson='''{
    "iter_enabled": false,
    "iter_list_is_const": false,
    "iter_list": "checked_list",
    "iter_parameter": "resource_arn"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_attach_tags_to_resources, lego_printer=aws_attach_tags_to_resources_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to attach tags. This Runbook gets the list of all untagged resources of a given region, discovers tag keys of the given region and attaches mandatory tags to all the untagged resource. To view the full platform capabilities of unSkript please visit https://unskript.com